In [2]:
import numpy as np
import pandas as pd

In [4]:
data_dir = './data/raw/participants/'
test_news = pd.read_csv(data_dir + 'train_news.csv')
train_news = pd.read_csv(data_dir + "train_news.csv")
train_candles = pd.read_csv(data_dir + 'train_candles.csv')

In [5]:
train_candles['begin'] = pd.to_datetime(train_candles['begin'])
train_candles.sort_values(by='begin', inplace=True)
train_candles['diff'] = train_candles.groupby(by='ticker')['begin'].diff().dt.days
train_candles['weekday'] = train_candles['begin'].dt.weekday

train_candles

,open,close,high,low,volume,begin,ticker,target_return_1d,target_direction_1d,target_return_20d,target_direction_20d,diff,weekday
0,81.50,81.70,83.20,81.16,29755530,2020-06-19,AFLT,0.004896,1,0.071726,1,NaN,4
20706,1344.00,1335.40,1355.20,1329.60,106126,2020-06-19,T,0.027258,1,0.289951,1,NaN,4
18274,205.44,207.00,209.20,204.55,75045510,2020-06-19,SBER,-0.002899,0,0.042174,1,NaN,4
4876,190.50,190.04,194.14,189.20,393392,2020-06-19,GMKN,0.006209,1,-0.032835,0,NaN,4
9748,116.18,115.67,116.50,115.10,13055000,2020-06-19,MOEX,0.002853,1,0.077289,1,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17058,449.50,452.05,454.00,446.50,2455806,2025-04-15,ROSN,0.000442,1,-0.018029,0,1.0,1
10966,193.18,191.81,194.00,190.31,5548830,2025-04-15,MOEX,0.013868,1,0.037120,1,1.0,1
4875,132.00,131.60,134.30,129.74,80694900,2025-04-15,GAZP,0.003040,1,0.085866,1,1.0,1
2437,48.42,48.97,49.27,48.25,15293730,2025-04-15,ALRS,0.010415,1,-0.008781,0,1.0,1


In [84]:
df.columns

Index(['open', 'close', 'high', 'low', 'volume', 'begin', 'ticker',
       'target_return_1d', 'target_direction_1d', 'target_return_20d',
       'target_direction_20d', 'diff', 'weekday'],
      dtype='object')

In [90]:
import pandas as pd

def make_ml_dataset(df, 
                    ticker: str,
                    ts_cols: list,       # временные фичи, которые разворачиваем по окну
                    single_cols: list,   # "одноразовые" колонки (например таргеты)
                    window: int = 5      # размер окна
                   ) -> pd.DataFrame:
    """
    Формирует датасет для обучения из временного ряда по конкретному тикеру.
    
    Args:
        df : DataFrame — исходные данные (содержит все тикеры)
        ticker : str — тикер, по которому строим выборку
        ts_cols : list — список колонок временного ряда (разворачиваем по окну)
        single_cols : list — список колонок, которые оставляем в одном экземпляре (кроме begin)
        window : int — размер окна (кол-во шагов)
    
    Returns:
        DataFrame для обучения
    """
    
    # фильтруем по тикеру
    data = df[df["ticker"] == ticker].reset_index(drop=True)
    
    X_list = []
    for i in range(window, len(data)):
        row = {}
        
        # 1. временные признаки по окну
        for j in range(window):
            shift = window - j - 1
            for col in ts_cols:
                row[f"{col}{j+1}"] = data.loc[i - shift, col]
        
        # 2. begin / end для окна
        begin_window = data.loc[i - window + 1 : i, "begin"].min()
        end_window   = data.loc[i - window + 1 : i, "begin"].max()
        row["begin"] = begin_window
        row["end"]   = end_window
        
        # 3. одноразовые колонки (например таргеты)
        for col in single_cols:
            if col != "begin":   # begin мы уже пересчитали
                row[col] = data.loc[i, col]
        
        X_list.append(row)
    
    return pd.DataFrame(X_list)


In [1]:
# Список колонок временного ряда
ts_cols = ["open", "close", "high", "low", "volume", "weekday"]

single_cols = ["begin", "target_return_1d", "target_direction_1d", 
               "target_return_20d", "target_direction_20d"]

train_df = make_ml_dataset(train_candles, 
                           ticker="T", 
                           ts_cols=ts_cols, 
                           single_cols=single_cols,
                           window=3)
train_df

NameError: name 'make_ml_dataset' is not defined

In [75]:
train_news['publication'][1]

'Тенденции в отрасли. Ключевые российские сталелитейные компании сохраняют конкурентоспособность на операционном и финансовом уровне относительно аналогов. Высокая рентабельность сочетается с низкой долговой нагрузкой, стабильными денежными потоками и высокой дивидендной доходностью. Прошедший год был неблагоприятным с точки зрения ценовой конъюнктуры, но высокая премия на внутреннем рынке сыграла в плюс, поэтому существенно просел экспорт. Учитывая циклический характер развития отрасли и замедление промышленного производства, финансовые результаты остались на уровне прошлого года или ухудшились, что оказало давление на котировки. Положение относительно конкурентов  Источник: Bloomberg: ПСБ Аналитика & Стратегия Размер шара – Funds form Operation, млрд долл. Корпоративные события. Из отраслевых событий отметим введение пошлин на импорт г/к проката, а также запрет на экспорт лома. В период до 2023г. рост спроса на металлургическую продукцию должна обеспечить реализация национальных прое

0       2020-06-19
1       2020-06-22
2       2020-06-23
3       2020-06-25
4       2020-06-26
           ...    
23112   2025-04-10
23113   2025-04-11
23114   2025-04-12
23115   2025-04-13
23116   2025-04-14
Name: begin, Length: 23117, dtype: datetime64[ns]